<a href="https://colab.research.google.com/github/Rainniee/Neural-Networks-AI/blob/master/Assignment_2_1_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load tiny-imagenet-200

In [1]:
import os, zipfile, io, requests
URL = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
def download_images(url):
    r = requests.get(url, stream=True)
    print ('Downloading ' + url )
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    zip_ref.extractall('./')
    zip_ref.close()
download_images(URL)



```
Tiny ImageNet contains 200 image classes
Each class has 500 training images, 50 validation images, and 50 test images 

1.   training dataset of 100,000 images
2.   validation dataset of 10,000 images
3.   test dataset of 10,000 images

```



### Split the training and validation dataset

In [2]:
import numpy as np
from PIL import Image
import six.moves.cPickle as pickle

data = {}
data['train'] = {}
data['val'] = {}
data['train']['image'] = []
data['train']['label'] = []
data['val']['image'] = []
data['val']['label'] = []

size = (32, 32)

N = 400 
## 80% as training dataset and 20% as validation dataset in each 500 images

# First load wnids
wnids = list(map(lambda x: x.strip(), open('tiny-imagenet-200/wnids.txt').readlines()))

# Split the training and validation dataset
for i in range(len(wnids)):
    wnid = wnids[i]
    print ("{}: {} / {}".format(wnid, i + 1, len(wnids)))
    
    for j in range(500):
        path = "tiny-imagenet-200/train/{0}/images/{0}_{1}.JPEG".format(wnid, j)
        image = (Image.open(path).convert('RGB'))
        image = image.resize(size, Image.ANTIALIAS)
        image = np.array(image)
        if j < N:
            data['train']['image'].append(image)
            data['train']['label'].append(i)
        else:
            data['val']['image'].append(image)
            data['val']['label'].append(i)

# Dump train.pkl
pickle.dump(data, open('train.pkl', 'wb', -1))

n02124075: 1 / 200
n04067472: 2 / 200
n04540053: 3 / 200
n04099969: 4 / 200
n07749582: 5 / 200
n01641577: 6 / 200
n02802426: 7 / 200
n09246464: 8 / 200
n07920052: 9 / 200
n03970156: 10 / 200
n03891332: 11 / 200
n02106662: 12 / 200
n03201208: 13 / 200
n02279972: 14 / 200
n02132136: 15 / 200
n04146614: 16 / 200
n07873807: 17 / 200
n02364673: 18 / 200
n04507155: 19 / 200
n03854065: 20 / 200
n03838899: 21 / 200
n03733131: 22 / 200
n01443537: 23 / 200
n07875152: 24 / 200
n03544143: 25 / 200
n09428293: 26 / 200
n03085013: 27 / 200
n02437312: 28 / 200
n07614500: 29 / 200
n03804744: 30 / 200
n04265275: 31 / 200
n02963159: 32 / 200
n02486410: 33 / 200
n01944390: 34 / 200
n09256479: 35 / 200
n02058221: 36 / 200
n04275548: 37 / 200
n02321529: 38 / 200
n02769748: 39 / 200
n02099712: 40 / 200
n07695742: 41 / 200
n02056570: 42 / 200
n02281406: 43 / 200
n01774750: 44 / 200
n02509815: 45 / 200
n03983396: 46 / 200
n07753592: 47 / 200
n04254777: 48 / 200
n02233338: 49 / 200
n04008634: 50 / 200
n02823428

#### Preprossing the training and validation

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_val, y_val) = (data['train']['image'],data['train']['label']),(data['val']['image'],data['val']['label'])

num_classes=200

x_train = np.array(x_train)
x_val = np.array(x_val)
x_train =x_train/ 255
x_val = x_val/255
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

Using TensorFlow backend.


### Building the model and modify the optimizer

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
from time import time
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.callbacks import TensorBoard, EarlyStopping

opts = [('SGD', SGD()), ('Adagrad', Adagrad()), ('Adadelta', Adadelta()), 
        ('Adam', Adam()), ('Adamax', Adamax())]

batch_size = 32
num_epochs = 50
data_augmentation = True

num_predictions = 20
Accuracy_list = []

save_dir = os.path.join(os.getcwd(), 'saved_models')

for name,opt in opts:
    print('Training ' + name + ' optimizer')
    logs = "logs/optimizer/"+name
    
    tensorboard = TensorBoard(log_dir=logs)    
    model_name = name + '_Keras_Optmizer_TinyImagenet200.h5'

  # Ttrain the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,metrics=['accuracy'])

    start = time()
    
    if not data_augmentation:
      print('Not using data augmentation.') 
      model.fit(x_train, y_train,
                batch_size=batch_size,
			          epochs=num_epochs,
			          validation_data=(x_val, y_val),
			          shuffle=True)
  
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        history=model.fit_generator(datagen.flow(x_train, y_train,
                                                 batch_size=batch_size),
                                    epochs=num_epochs,
                                    steps_per_epoch=500,
                                    workers = 4,
                                    validation_data=(x_val, y_val)
                                    )
        
        
    # Save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(x_val, y_val, verbose=1)

    end = time()
    print('Time taken to run:', str(end-start))

    print('Validation loss:', scores[0])
    print('Validation accuracy:', scores[1])
    
    Accuracy_list.append([name,scores[1]])
        

Training SGD optimizer
Using real-time data augmentation.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
500/500 [==============================] - 22s 44ms/step - loss: 5.2988 - acc: 0.0057 - val_loss: 5.2980 - val_acc: 0.0048
Epoch 2/50
500/500 [==============================] - 17s 34ms/step - loss: 5.2985 - acc: 0.0055 - val_loss: 5.2975 - val_acc: 0.0069
Epoch 3/50
500/500 [==============================] - 17s 34ms/step - loss: 5.2979 - acc: 0.0054 - val_loss: 5.2969 - val_acc: 0.0053
Epoch 4/50
500/500 [==============================] - 17s 34ms/step - loss: 5.2975 - acc: 0.0056 - val_loss: 5.2957 - val_acc: 0.0062
Epoch 5/50
500/500 [==============================] - 17s 34ms/step - loss: 5.2957 - acc: 0.0057 - val_loss: 5.2926 - val_acc: 0.0069
Epoch 6/50
500/500 [==============================] - 17s 34ms/step - loss: 5.2909 - acc: 0.0063 - val_loss: 5.2837 - val_acc: 0.0085
Epoch 7/50
500/500 [==============================] - 18s 36ms/step - loss: 5.2790 - acc: 

In [0]:
Accuracy_list

[['SGD', 0.09705],
 ['Adagrad', 0.21495],
 ['Adadelta', 0.23295],
 ['Adam', 0.22385],
 ['Adamax', 0.25495]]

*   Adamax resulted in the max accuracy



### Building the model and modify the activation

In [0]:
activations = ['relu','elu','tanh']

batch_size = 32
num_epochs = 50
data_augmentation = True

num_predictions = 20
Accuracy_list = []

save_dir = os.path.join(os.getcwd(), 'saved_models')

for acti in activations:
  print('Training' + acti + ' Activation function')
  
  logs = "logs/activation/"+name
    
  model_name = acti + '_Keras_Activation_TinyImagenet200.h5'
  
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation(acti))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation(acti))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation(acti))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation(acti))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation(acti))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation(acti))

  model.summary()
  
  model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

  start = time()
    
  if not data_augmentation:
    print('Not using data augmentation.') 
    model.fit(x_train, y_train,
              batch_size=batch_size,
			        epochs=num_epochs,
			        validation_data=(x_val, y_val),
			        shuffle=True)
  
  else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history=model.fit_generator(datagen.flow(x_train, y_train,
                                             batch_size=batch_size),
                                epochs=num_epochs,
                                steps_per_epoch=500,
                                workers = 4,
                                validation_data=(x_val, y_val)
                                )
        
        
  # Save model and weights
  if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(x_val, y_val, verbose=1)

    end = time()
    print('Time taken to run:', str(end-start))

    print('Validation loss:', scores[0])
    print('Validation accuracy:', scores[1])
    
    Accuracy_list.append([name,scores[1]])

Trainingrelu Activation function
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15,



*   elu activation resulted in the best performance



### Building the model and modify the batch size

#### Btach size [32, 64, 128]

In [0]:
batchsize = [32, 64, 128, 512]

Accuracy_list = []

num_classes = 200
num_epochs = 50
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')

for bat in batchsize:
  print('Training' + str(bat) + ' Batchsize')
  
  logs = "logs/batch_size/"+str(bat)     
  model_name = str(bat) + '_Keras_Batchsize_TinyImagenet200.h5'
  
  batch_size = bat
  
  data_augmentation = True
  
  start = time()
    
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation('elu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('elu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  model.summary()
  
  model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

  start = time()
    
  if not data_augmentation:
    print('Not using data augmentation.') 
    model.fit(x_train, y_train,
              batch_size=batch_size,
			        epochs=num_epochs,
			        validation_data=(x_val, y_val),
			        shuffle=True)
  
  else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history=model.fit_generator(datagen.flow(x_train, y_train,
                                             batch_size=batch_size),
                                epochs=num_epochs,
                                steps_per_epoch=500,
                                workers = 4,
                                validation_data=(x_val, y_val)
                                )
        
        
  # Save model and weights
  if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(x_val, y_val, verbose=1)

    end = time()
    print('Time taken to run:', str(end-start))

    print('Validation loss:', scores[0])
    print('Validation accuracy:', scores[1])
    
    Accuracy_list.append([str(bat),scores[1]])

Training32 Batchsize
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        

#### Btach size [512]

In [0]:
from time import time
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.callbacks import TensorBoard, EarlyStopping

batchsize = [512]
Accuracy_list = []

num_classes = 200
num_epochs = 50
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')

for bat in batchsize:
  print('Training' + str(bat) + ' Batchsize')
  
  logs = "logs/batch_size/"+str(bat)     
  model_name = str(bat) + '_Keras_Batchsize_TinyImagenet200.h5'
  
  batch_size = bat
  
  data_augmentation = True
  
  start = time()
    
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation('elu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('elu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  model.summary()
  
  model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

  start = time()
    
  if not data_augmentation:
    print('Not using data augmentation.') 
    model.fit(x_train, y_train,
              batch_size=batch_size,
			        epochs=num_epochs,
			        validation_data=(x_val, y_val),
			        shuffle=True)
  
  else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history=model.fit_generator(datagen.flow(x_train, y_train,
                                             batch_size=batch_size),
                                epochs=num_epochs,
                                steps_per_epoch=500,
                                workers = 4,
                                validation_data=(x_val, y_val)
                                )
        
        
  # Save model and weights
  if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(x_val, y_val, verbose=1)

    end = time()
    print('Time taken to run:', str(end-start))

    print('Validation loss:', scores[0])
    print('Validation accuracy:', scores[1])
    
    Accuracy_list.append([str(bat),scores[1]])

Training512 Batchsize
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________

### Hidden Layers & Neurons

In [4]:
from time import time
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.callbacks import TensorBoard, EarlyStopping

batch_size = 512
num_classes = 200
num_epochs = 50
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')

print('Training 3 layers')
  
logs = "logs/3layers/"     
model_name = 'Keras_3layers_TinyImagenet200.h5'
  
data_augmentation = True
start = time()
  
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', 
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
          
model.summary()
          
model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

start = time()
    
if not data_augmentation:
  print('Not using data augmentation.') 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=num_epochs,
			      validation_data=(x_val, y_val),
			      shuffle=True)
  
else:
  print('Using real-time data augmentation.')
  # This will do preprocessing and realtime data augmentation:
  datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
      samplewise_center=False,  # set each sample mean to 0
      featurewise_std_normalization=False,  # divide inputs by std of the dataset
      samplewise_std_normalization=False,  # divide each input by its std
      zca_whitening=False,  # apply ZCA whitening
      zca_epsilon=1e-06,  # epsilon for ZCA whitening
      rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
      # randomly shift images horizontally (fraction of total width)
      width_shift_range=0.1,
      # randomly shift images vertically (fraction of total height)
      height_shift_range=0.1,
      shear_range=0.,  # set range for random shear
      zoom_range=0.,  # set range for random zoom
      channel_shift_range=0.,  # set range for random channel shifts
      # set mode for filling points outside the input boundaries
      fill_mode='nearest',
      cval=0.,  # value used for fill_mode = "constant"
      horizontal_flip=True,  # randomly flip images
      vertical_flip=False,  # randomly flip images
      # set rescaling factor (applied before any other transformation
      rescale=None,
      # set function that will be applied on each input
      preprocessing_function=None,
      # image data format, either "channels_first" or "channels_last"
      data_format=None,
      # fraction of images reserved for validation (strictly between 0 and 1)
      validation_split=0.0)

  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)

  # Fit the model on the batches generated by datagen.flow().
  history=model.fit_generator(datagen.flow(x_train, y_train,
                                           batch_size=batch_size),
                              epochs=num_epochs,
                              steps_per_epoch=500,
                              workers = 4,
                              validation_data=(x_val, y_val)
                              )
        
        
# Save model and weights
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)    
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path)
  print('Saved trained model at %s ' % model_path)
    
  # Score trained model.
  scores = model.evaluate(x_val, y_val, verbose=1)

  end = time()
  print('Time taken to run:', str(end-start))

  print('Validation loss:', scores[0])
  print('Validation accuracy:', scores[1])
    

Training 3 layers
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dro

In [0]:
batch_size = 512
num_classes = 200
num_epochs = 50
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')

print('Training 4 layers')
  
logs = "logs/4layers/"     
model_name = 'Keras_4layers_TinyImagenet200.h5'
  
data_augmentation = True
start = time()

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', 
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(1024, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(2048)
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
          
model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

start = time()
    
if not data_augmentation:
  print('Not using data augmentation.') 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=num_epochs,
			      validation_data=(x_val, y_val),
			      shuffle=True)
  
else:
  print('Using real-time data augmentation.')
  # This will do preprocessing and realtime data augmentation:
  datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
      samplewise_center=False,  # set each sample mean to 0
      featurewise_std_normalization=False,  # divide inputs by std of the dataset
      samplewise_std_normalization=False,  # divide each input by its std
      zca_whitening=False,  # apply ZCA whitening
      zca_epsilon=1e-06,  # epsilon for ZCA whitening
      rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
      # randomly shift images horizontally (fraction of total width)
      width_shift_range=0.1,
      # randomly shift images vertically (fraction of total height)
      height_shift_range=0.1,
      shear_range=0.,  # set range for random shear
      zoom_range=0.,  # set range for random zoom
      channel_shift_range=0.,  # set range for random channel shifts
      # set mode for filling points outside the input boundaries
      fill_mode='nearest',
      cval=0.,  # value used for fill_mode = "constant"
      horizontal_flip=True,  # randomly flip images
      vertical_flip=False,  # randomly flip images
      # set rescaling factor (applied before any other transformation
      rescale=None,
      # set function that will be applied on each input
      preprocessing_function=None,
      # image data format, either "channels_first" or "channels_last"
      data_format=None,
      # fraction of images reserved for validation (strictly between 0 and 1)
      validation_split=0.0)

  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)

  # Fit the model on the batches generated by datagen.flow().
  history=model.fit_generator(datagen.flow(x_train, y_train,
                                           batch_size=batch_size),
                              epochs=num_epochs,
                              steps_per_epoch=500,
                              workers = 4,
                              validation_data=(x_val, y_val)
                              )
        
        
# Save model and weights
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)    
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path)
  print('Saved trained model at %s ' % model_path)
    
  # Score trained model.
  scores = model.evaluate(x_val, y_val, verbose=1)

  end = time()
  print('Time taken to run:', str(end-start))

  print('Validation loss:', scores[0])
  print('Validation accuracy:', scores[1])

### Trying Batch Normalization

In [0]:
from keras.layers.normalization import BatchNormalization

batch_size = 512
num_classes = 200
num_epochs = 50
num_predictions = 20

save_dir = os.path.join(os.getcwd(), 'saved_models')

print('Training with batch normalization')
  
logs = "logs/batch_normalization/"     
model_name = 'Keras_normalization_TinyImagenet200.h5'
  
data_augmentation = True
start = time()

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', 
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(1024, (3, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(2048)
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
          
model.compile(loss='categorical_crossentropy', optimizer=Adamax(), metrics=['accuracy'])

start = time()
    
if not data_augmentation:
  print('Not using data augmentation.') 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=num_epochs,
			      validation_data=(x_val, y_val),
			      shuffle=True)
  
else:
  print('Using real-time data augmentation.')
  # This will do preprocessing and realtime data augmentation:
  datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
      samplewise_center=False,  # set each sample mean to 0
      featurewise_std_normalization=False,  # divide inputs by std of the dataset
      samplewise_std_normalization=False,  # divide each input by its std
      zca_whitening=False,  # apply ZCA whitening
      zca_epsilon=1e-06,  # epsilon for ZCA whitening
      rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
      # randomly shift images horizontally (fraction of total width)
      width_shift_range=0.1,
      # randomly shift images vertically (fraction of total height)
      height_shift_range=0.1,
      shear_range=0.,  # set range for random shear
      zoom_range=0.,  # set range for random zoom
      channel_shift_range=0.,  # set range for random channel shifts
      # set mode for filling points outside the input boundaries
      fill_mode='nearest',
      cval=0.,  # value used for fill_mode = "constant"
      horizontal_flip=True,  # randomly flip images
      vertical_flip=False,  # randomly flip images
      # set rescaling factor (applied before any other transformation
      rescale=None,
      # set function that will be applied on each input
      preprocessing_function=None,
      # image data format, either "channels_first" or "channels_last"
      data_format=None,
      # fraction of images reserved for validation (strictly between 0 and 1)
      validation_split=0.0)

  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)

  # Fit the model on the batches generated by datagen.flow().
  history=model.fit_generator(datagen.flow(x_train, y_train,
                                           batch_size=batch_size),
                              epochs=num_epochs,
                              steps_per_epoch=500,
                              workers = 4,
                              validation_data=(x_val, y_val)
                              )
        
        
# Save model and weights
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)    
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path)
  print('Saved trained model at %s ' % model_path)
    
  # Score trained model.
  scores = model.evaluate(x_val, y_val, verbose=1)

  end = time()
  print('Time taken to run:', str(end-start))

  print('Validation loss:', scores[0])
  print('Validation accuracy:', scores[1])